In [2]:
import pandas as pd
from preprocess import getPopularityIndex
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from preprocess import cleanData, getPopularityIndex, encodePandas
from sklearn.preprocessing import OneHotEncoder, StandardScaler


In [9]:
X = pd.read_pickle('TwitterData_1e5_90rm.pkl')
y = getPopularityIndex(X, 4)
X = cleanData(X)
print('X dataframe has {} rows...'.format(len(X)))
numerical_columns = X._get_numeric_data().columns
categorical_columns = list(set(X.columns) - set(numerical_columns))
print('...with numerical columns:')
print(numerical_columns)
print('and categorical columns:')
print(categorical_columns)

print('\n')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Trainig set contains {} samples'.format(len(X_train)))
print('Test set contains {} samples'.format(len(X_test)))

X dataframe has 20467 rows...
...with numerical columns:
Index(['followers_count', 'friends_count', 'listed_count', 'statuses_count',
       'mentions_tot_followers', 'mentions_tot_friends',
       'mentions_tot_statuses', 'n_sentences', 'n_words', 'n_adjectives',
       'n_adverbs', 'n_nouns', 'n_pronouns', 'n_verbs', 'n_long_words',
       'n_hashtags', 'n_user_mentions'],
      dtype='object')
and categorical columns:
['has_gif', 'has_photo', 'has_questions', 'has_symbols', 'has_urls', 'day', 'is_reply', 'has_video', 'time']


Trainig set contains 16373 samples
Test set contains 4094 samples


In [11]:
categorical_preprocessor = OneHotEncoder(handle_unknown='ignore', drop='first')
numerical_preprocessor = StandardScaler()

preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard_scaler', numerical_preprocessor, numerical_columns)
    ])

In [15]:
model = make_pipeline(preprocessor, SVC(gamma='auto'))

In [16]:
model.fit(X_train, y_train)
print(model.score(X_train, y_train))

0.5889574299151041
